## 3.4 엑셀 통합 문서에서 워크시트 집합 읽기

### 3.4.1 통합 문서에서 워크시트의 일부분만 처리할 때

In [177]:
ls

11output.xls                        3output.xls
11pandas_output.xls                 3장.ipynb
1excel_introspect_workbook.ipynb    4-2output.xls
1excel_introspect_workbook.py       4excel_value_meets_condition.ipynb
2excel_parsing_and_write.ipynb      4output.xls
2output.xls                         4pandas_output.xls
3.4.md                              sales_2013.xlsx
3.4~3.6.ipynb                       sales_2014.xlsx
3.5.md                              sales_2015.xlsx


In [176]:
import sys
from datetime import date
from xlrd import open_workbook, xldate_as_tuple
from xlwt import Workbook

input_file = "./sales_2013.xlsx"
output_file = "11output.xls"

output_workbook = Workbook() #데이터를 쌓을 빈 워크북을 만든다
output_worksheet = output_workbook.add_sheet('set_of_worksheets') 
#위의 워크북에 'set_of_worksheets'라는 이름의 워크시트를 추가한다

my_sheets = [0,1]
threshold = 1900.0
sales_column_index = 3


first_worksheet = True #첫 번째 워크시트가 참인 상태
with open_workbook(input_file) as workbook:
    data = [] #아래 for문을 통해 필터링된 데이터를 추가할 빈 리스트
    for sheet_index in range(workbook.nsheets): #가져온 엑셀 파일에 있는 워크시트 중에서
        if sheet_index in my_sheets: #(my_sheets = [0,1])범위 안에서
            worksheet = workbook.sheet_by_index(sheet_index) #워크시트 위치를 기준으로 가져온다.
            if first_worksheet: #그리고 첫 번째 워크시트 = True일 때
                header_row = worksheet.row_values(0) #워크시트의 0번 row_value(첫 번째 행)을 header_row로 할당한다.
                data.append(header_row) #header_row를 위의 데이터 리스트에 추가한다
                first_worksheet = False #헤더를 가져오는 작업이 끝났으므로 상태를 변경한다.
                
                #print("header_row:",header_row)
                #print("data:",data)
                
            for row_index in range(1,worksheet.nrows): #헤더 다음 행부터
                row_list = []
                sale_amount = worksheet.cell_value(row_index, sales_column_index)
                #빈 row_list와 sale_amount를 가져올 셀 위치(행, 열) 지정
                if sale_amount > threshold:
                    for column_index in range(worksheet.ncols):
                        cell_value = worksheet.cell_value(row_index,column_index) 
                        #sale_amount가 1900 초과할 때, cell_value에 행,열 위치를 기록한다
                        cell_type = worksheet.cell_type(row_index,column_index) 
                        #각 위치의 데이터 타입을 기록한다
                        if cell_type == 3: #type 3은 날짜
                            date_cell = xldate_as_tuple(cell_value,workbook.datemode)
                            date_cell = date(*date_cell[0:3]).strftime('%m/%d/%Y')
                            #데이터 타입이 날짜일 때, 월/일/연 형태의 날짜 데이터로 변환해 date_cell에 기록한다
                            row_list.append(date_cell) #date_cell을 row_list에 추가한다.
                        else:
                            row_list.append(cell_value) #날짜가 아닌 셀 값을 row_list에 추가한다.
                    if row_list:
                        data.append(row_list)
                        #유효한 row_list를 data 리스트에 추가한다
    print(data)

    print("\n행, 열, 내용 : ")
    for list_index, output_list in enumerate(data):
        for element_index, element in enumerate(output_list):
            output_worksheet.write(list_index, element_index, element)
            print(list_index, element_index, element)
    
output_workbook.save(output_file)

[['Customer ID', 'Customer Name', 'Invoice Number', 'Sale Amount', 'Purchase Date'], [6789.0, 'Samantha Donaldson', '100-0007', 1995.0, '01/31/2013'], [7654.0, 'Roger Lipney', '100-0010', 2135.0, '02/15/2013']]

행, 열, 내용 : 
0 0 Customer ID
0 1 Customer Name
0 2 Invoice Number
0 3 Sale Amount
0 4 Purchase Date
1 0 6789.0
1 1 Samantha Donaldson
1 2 100-0007
1 3 1995.0
1 4 01/31/2013
2 0 7654.0
2 1 Roger Lipney
2 2 100-0010
2 3 2135.0
2 4 02/15/2013


#### 팬더스

- read_excel() 함수에서 인덱스 번호나 이름으로 워크시트 선택

In [171]:
import pandas as pd

input_file = "./sales_2013.xlsx"
output_file = "11pandas_output.xls"

my_sheets = [0,1]
threshold = 1900.0

data_frame = pd.read_excel(input_file, sheetname=my_sheets, index_col=None)

row_list = []
for worksheet_name, data in data_frame.items():
    row_list.append(data[data['Sale Amount'].astype(float) > threshold])
    # data_frame 안의 워크시트 순서대로 데이터를 가져오고 float타입으로 변환한 'Sale Amount' 값에 따라 row_list에 추가함
print(worksheet_name, row_list)

filtered_rows = pd.concat(row_list, axis=0, ignore_index=True)
#필터링된 행을 하나의 데이터로 합침

writer = pd.ExcelWriter(output_file)
filtered_rows.to_excel(writer, sheet_name='set_of_worksheets',index=False)
writer.save()


1 [   Customer ID       Customer Name Invoice Number  Sale Amount Purchase Date
5         6789  Samantha Donaldson       100-0007         1995    2013-01-31,    Customer ID Customer Name Invoice Number  Sale Amount Purchase Date
2         7654  Roger Lipney       100-0010         2135    2013-02-15]


## 3.5 여러 개의 통합문서 처리하기
### 3.5.1 통합 문서의 개수 및 각 통합 문서의 행과 열 개수 세기

In [197]:
import glob
import os
from xlrd import open_workbook

input_directory = "./"

workbook_counter = 0
for input_file in glob.glob(os.path.join(input_directory, '*.xlsx')):
    workbook = open_workbook(input_file)
    print('Workbook: {}'.format(os.path.basename(input_file)))
    print('number of worksheets: {}'.format(workbook.nsheets))
    for worksheet in workbook.sheets():
        print('Worksheet name:', worksheet.name, '\tRows',\
             worksheet.nrows, '\tColumns:', worksheet.ncols)
    print('\n')
    workbook_counter += 1
print('Number of Excel workbooks: {}'.format(workbook_counter))

Workbook: sales_2013.xlsx
number of worksheets: 3
Worksheet name: january_2013 	Rows 7 	Columns: 5
Worksheet name: february_2013 	Rows 7 	Columns: 5
Worksheet name: march_2013 	Rows 7 	Columns: 5


Workbook: sales_2014.xlsx
number of worksheets: 3
Worksheet name: january_2014 	Rows 7 	Columns: 5
Worksheet name: february_2014 	Rows 7 	Columns: 5
Worksheet name: march_2014 	Rows 7 	Columns: 5


Workbook: sales_2015.xlsx
number of worksheets: 3
Worksheet name: january_2015 	Rows 7 	Columns: 5
Worksheet name: february_2015 	Rows 7 	Columns: 5
Worksheet name: march_2015 	Rows 7 	Columns: 5


Number of Excel workbooks: 3


### 3.5.2 여러 개의 통합 문서 합치기

#### 기본 파이썬

In [189]:
import glob
import os
from datetime import date
from xlrd import open_workbook, xldate_as_tuple
from xlwt import Workbook

input_folder = "./"
output_file = "12output.xls"

output_workbook = Workbook()
output_worksheet = output_workbook.add_sheet('all_data_all_workbooks')

data = []
first_worksheet = True

for input_file in glob.glob(os.path.join(input_folder, '*.xlsx')):
    print(os.path.basename(input_file))

sales_2013.xlsx
sales_2014.xlsx
sales_2015.xlsx
